# Monte Carlo Explaination of Probability vs. Confidence
## Plus a copule bonuses -- the Bonferroni Inequalty, t-statistic/P-value

1/29/2019 - Jeff Smith

In [ ]:
# Inspired by https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def std_confidence_interval(data, confidence=.95):
    p = (1 - confidence)/2
    a = 1.0 * np.array(data)
    n = len(a)
    s = np.std(a)
    v = np.square(s)
    lcl = np.sqrt(((n-1)*v)/scipy.stats.chi2.ppf((1-p), n-1))
    ucl = np.sqrt(((n-1)*v)/scipy.stats.chi2.ppf(p, n-1))
    return s, lcl, ucl

In [ ]:
# Distribution parameters - Mean, Std Dev
mu = 225
sigma = 47
# Sample size
n = 100
# 1-alpha for the CI
c = .95

## C.I. on the Mean

In [ ]:
# Single Replication
data = np.random.normal(mu, sigma, n)
xbar, lcl, ucl = mean_confidence_interval(data, c)
print("xbar: {:.3f}, CI: ({:.3f}, {:.3f})".format(xbar, lcl, ucl))

In [ ]:
# Monte Carlo
reps = 1000
success = 0
means = []
for j in range(reps):
    data = np.random.normal(mu, sigma, n)
#    data = np.random.uniform(mu-sigma, mu+sigma, n)
#    data = np.random.exponential(mu, n)
    xbar, lcl, ucl = mean_confidence_interval(data,c)
    means.append(xbar)
    if lcl <= mu and ucl >= mu:
        success += 1
print("Pr. of success = {:.3f}".format(success/reps))

In [ ]:
plt.hist(means);

## C.I. on the Variance

In [ ]:
data = np.random.normal(mu, sigma, n)
s, slcl, sucl = std_confidence_interval(data, c)
print("s: {:.3f}, CI: ({:.3f}, {:.3f})".format(s, slcl, sucl))

In [ ]:
# Monte Carlo
reps = 1000
success = 0
stds = []
for j in range(reps):
    data = np.random.normal(mu, sigma, n)
    s, lcl, ucl = std_confidence_interval(data,c)
    stds.append(s)
    if lcl <= sigma and ucl >= sigma:
        success += 1
print("Pr. of success = {:.3f}".format(success/reps))

In [ ]:
plt.hist(stds);

## Bonferroni Inequality

In [ ]:
# Monte Carlo
reps = 1000
sm = 0
ss = 0
sb = 0
for j in range(reps):
    b = 0
    data = np.random.normal(mu, sigma, n)
    xbar, lcl, ucl = mean_confidence_interval(data,c)
    if lcl <= mu and ucl >= mu:
        sm += 1
        b = 1
    s, lcl, ucl = std_confidence_interval(data,c)
    if lcl <= sigma and ucl >= sigma:
        ss += 1
    else:
        b = 0
    sb += b
print("Mean - Pr. of success = {:.3f}".format(sm/reps))
print(" Std - Pr. of success = {:.3f}".format(ss/reps))
print("Both - Pr. of success = {:.3f}".format(sb/reps))

# t statistic and P-values

In [ ]:
# t_0 = (x-bar-mu) / (s/sqrt(n)) -- should follow a t distribution 
# with n-1 dof if the sample is either large or follows a normal
# distribution.
# Monte Carlo
reps = 100000
n = 100
sn = np.sqrt(n)
ts = []
for j in range(reps):
    data = np.random.normal(mu, sigma, n)
    ts.append((np.mean(data)-mu)/(np.std(data)/sn))
plt.hist(ts,bins=100);

In [ ]:
# 1-sample t-test on a random variate sample
t, p = scipy.stats.ttest_1samp(np.random.normal(mu, sigma, n), mu)
t, p